In [2]:
import pandas as pd

# Paths
ICU_PATH = "/kaggle/input/neuro-icu/icu/icu/"
DNN_PATH = "/kaggle/input/dnn-part/"

# --- Load base files ---
icu = pd.read_csv(
    ICU_PATH + "icustays.csv",
    usecols=["stay_id", "hadm_id", "subject_id", "first_careunit", "los"]
)
patients = pd.read_csv(
    DNN_PATH + "patients.csv",
    usecols=["subject_id", "gender", "anchor_age"]
)
admissions = pd.read_csv(
    DNN_PATH + "admissions.csv",
    usecols=["hadm_id", "admittime", "dischtime"]
)

# --- Merge static demographic info ---
static_df = (
    icu.merge(patients, on="subject_id", how="left")
       .merge(admissions, on="hadm_id", how="left")
)

# --- Load your engineered features + label ---
labels = pd.read_csv("/kaggle/input/static-features-dataset/model_neuro6h_scaled_full (1).csv")
static_df = static_df.merge(
    labels[["subject_id", "hadm_id", "ventilation_within_12h"]],
    on=["subject_id", "hadm_id"],
    how="left",
    suffixes=("", "_dup")
)

# --- Clean duplicate columns ---
for col in static_df.columns:
    if col.endswith("_dup"):
        base = col[:-4]
        if base in static_df.columns:
            static_df[base] = static_df[base].combine_first(static_df[col])
        static_df.drop(columns=[col], inplace=True)

# --- Rename columns ---
static_df.rename(columns={
    "anchor_age": "age",
    "ventilation_within_12h": "ventilated_12h"
}, inplace=True)

# --- Keep only the final DNN input columns ---
final_static = static_df[
    static_df["ventilated_12h"].notna()
][[
    "subject_id",
    "stay_id",
    "hadm_id",
    "age",
    "gender",
    "los",
    "first_careunit",
    "ventilated_12h"
]]

# Convert label to int (0/1)
final_static["ventilated_12h"] = final_static["ventilated_12h"].astype(int)



In [3]:
import pandas as pd

# Load engineered static dataset (scaled features + label)
static_df = pd.read_csv("/kaggle/input/static-features-dataset/model_neuro6h_scaled_full (1).csv")
dnn_old=pd.read_csv("/kaggle/input/dnn-old-file/static_features_labeled (1).csv")

# Load demographics from your neuro ICU master table
# Make sure neuro_icu is loaded before this step!
demo_df = dnn_old[['stay_id','subject_id','hadm_id','age','gender','first_careunit','los']].drop_duplicates()

# Merge on IDs
final_static = static_df.merge(
    demo_df,
    on=["subject_id","hadm_id"],
    how="left"
)

# Keep label at end
label = final_static['ventilation_within_12h']
final_static = final_static.drop(columns=['ventilation_within_12h'])
final_static['ventilation_within_12h'] = label.astype(int)

print("Shape:", final_static.shape)
print(final_static['ventilation_within_12h'].value_counts())


Shape: (8274, 25)
ventilation_within_12h
0    7085
1    1189
Name: count, dtype: int64


In [4]:
# Combine LOS columns (use los_x first, if missing use los_y)
final_static["los"] = final_static["los_x"].fillna(final_static["los_y"])

# Drop old LOS columns
final_static.drop(columns=["los_x", "los_y"], inplace=True)

# Desired first columns
front_cols = [
    "subject_id",
    "stay_id",
    "hadm_id",
    "age",
    "gender",
    "first_careunit",
    "los",
]

# Move these to front, keep rest in original order
remaining_cols = [c for c in final_static.columns if c not in front_cols]

final_static = final_static[front_cols + remaining_cols]

# Save
final_static.to_csv("static_features_final.csv", index=False)

print("Shape:", final_static.shape)
print(final_static['ventilation_within_12h'].value_counts())

print(final_static.head())


Shape: (8274, 24)
ventilation_within_12h
0    7085
1    1189
Name: count, dtype: int64
   subject_id   stay_id     hadm_id  age gender      first_careunit  \
0  10002348.0  32610785  22725460.0   77      F  Neuro Intermediate   
1  10004113.0  39774222  29879900.0   33      M      Neuro Stepdown   
2  10010399.0  38121508  24825793.0   54      F  Neuro Intermediate   
3  10011507.0  31950308  24996125.0   41      M  Neuro Intermediate   
4  10011938.0  36750867  28265343.0   57      F  Neuro Intermediate   

         los  vital_mean  vital_std  vital_min  ...  Ischemic_Stroke  \
0   9.792512   -0.088098  -0.043532   0.225534  ...              1.0   
1   1.060683   -0.027577  -0.042322   0.225534  ...              0.0   
2   2.294120   -0.070269  -0.029934   0.225534  ...              0.0   
3  11.761308   -0.028623  -0.005287  -0.277051  ...              0.0   
4  20.993843   -0.086247  -0.036761  -0.025759  ...              0.0   

   Hemorrhage  Aneurysm  Seizure  Brain_Tumor  Hydroc

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, classification_report

# --------- Load Data ---------
df = pd.read_csv("/kaggle/working/static_features_final.csv")
print("Loaded static dataset:", df.shape)

# --------- Preprocess ---------
cat_cols = ["gender", "first_careunit"]
num_cols = [
    "age", "los",
    "vital_mean", "vital_std", "vital_min", "vital_max",
    "vital_count", "total_input_ml", "total_output_ml"
] + [
    "Ischemic_Stroke","Hemorrhage","Aneurysm","Seizure","Brain_Tumor",
    "Hydrocephalus","AVM","Traumatic_Brain_Injury","Spinal_Cord"
]

df = df.copy()
df[cat_cols] = df[cat_cols].fillna("Unknown")
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
scaler = StandardScaler()

X_cat = encoder.fit_transform(df[cat_cols])
X_num = scaler.fit_transform(df[num_cols])
X = np.hstack([X_num, X_cat])
y = df["ventilation_within_12h"].values

# Split AFTER preprocessing
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
X_val_t = torch.tensor(X_val, dtype=torch.float32)
y_val_t = torch.tensor(y_val.reshape(-1, 1), dtype=torch.float32)

# --------- Model Definition ---------
class DNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.classifier = nn.Linear(32, 1)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        features = self.feature_extractor(x)
        logits = self.classifier(features)
        return logits, features

model = DNN(X_train.shape[1])
use_focal = False

# --------- Compute pos_weight (Now y_train exists) ---------
neg, pos = (y_train == 0).sum(), (y_train == 1).sum()
pos_weight = torch.tensor([neg / pos], dtype=torch.float32)
print(f"pos_weight = {pos_weight.item():.2f}")

# --------- Focal Loss Function ---------
def focal_loss(logits, targets, alpha=0.75, gamma=2):
    prob = torch.sigmoid(logits)
    bce = nn.functional.binary_cross_entropy_with_logits(
        logits, targets, reduction='none'
    )
    fl = alpha * (1 - prob)**gamma * bce
    return fl.mean()

# --------- Criterion, Optimizer, Scheduler ---------
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-6
)

# --------- Training Loop ---------
best_val_loss = np.inf
patience, patience_counter = 10, 0
epochs = 200

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    logits, _ = model(X_train_t)
    loss = focal_loss(logits, y_train_t) if use_focal else criterion(logits, y_train_t)

    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_logits, _ = model(X_val_t)
        val_loss = criterion(val_logits, y_val_t)
        scheduler.step(val_loss)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⏳ Early stopping triggered")
            break

model.load_state_dict(best_state)

# --------- Threshold Tuning ---------
model.eval()
with torch.no_grad():
    probs = torch.sigmoid(model(X_val_t)[0]).numpy().flatten()

prec, rec, thresholds = precision_recall_curve(y_val, probs)
f1_scores = 2 * (prec * rec) / (prec + rec + 1e-8)
best_idx = np.argmax(f1_scores)
best_thresh = thresholds[best_idx]

print(f"\n🎯 Best Threshold = {best_thresh:.3f}")
print(f"Precision={prec[best_idx]:.3f}, Recall={rec[best_idx]:.3f}, F1={f1_scores[best_idx]:.3f}")

final_preds = (probs > best_thresh).astype(int)

print("ROC-AUC:", roc_auc_score(y_val, probs))
print(classification_report(y_val, final_preds))


Loaded static dataset: (8274, 24)
pos_weight = 6.08
Epoch 001 | Loss: 1.5998 | Val Loss: 1.6017
Epoch 002 | Loss: 1.5453 | Val Loss: 1.5548
Epoch 003 | Loss: 1.5007 | Val Loss: 1.5109
Epoch 004 | Loss: 1.4778 | Val Loss: 1.4702
Epoch 005 | Loss: 1.4234 | Val Loss: 1.4328
Epoch 006 | Loss: 1.3856 | Val Loss: 1.3981
Epoch 007 | Loss: 1.3571 | Val Loss: 1.3658
Epoch 008 | Loss: 1.3304 | Val Loss: 1.3356
Epoch 009 | Loss: 1.2986 | Val Loss: 1.3077
Epoch 010 | Loss: 1.2835 | Val Loss: 1.2817
Epoch 011 | Loss: 1.2399 | Val Loss: 1.2577
Epoch 012 | Loss: 1.2291 | Val Loss: 1.2354
Epoch 013 | Loss: 1.2172 | Val Loss: 1.2146
Epoch 014 | Loss: 1.1938 | Val Loss: 1.1956
Epoch 015 | Loss: 1.1690 | Val Loss: 1.1781
Epoch 016 | Loss: 1.1602 | Val Loss: 1.1618
Epoch 017 | Loss: 1.1360 | Val Loss: 1.1468
Epoch 018 | Loss: 1.1247 | Val Loss: 1.1328
Epoch 019 | Loss: 1.1075 | Val Loss: 1.1195
Epoch 020 | Loss: 1.0942 | Val Loss: 1.1069
Epoch 021 | Loss: 1.0909 | Val Loss: 1.0948
Epoch 022 | Loss: 1.0796

In [6]:
# --------- Extract static embeddings from DNN (FULL DATASET) ---------

model.eval()

def get_embeddings(model, X):
    with torch.no_grad():
        _, features = model(torch.tensor(X, dtype=torch.float32))
    return features.numpy()

# Extract embeddings for ALL rows
static_embeddings = get_embeddings(model, X)

# Save to CSV
import pandas as pd

df_emb = pd.DataFrame(
    static_embeddings,
    columns=[f"static_emb_{i}" for i in range(static_embeddings.shape[1])]
)

df_emb = pd.concat([df[['stay_id']].reset_index(drop=True), df_emb], axis=1)
df_emb.to_csv("static_embeddings.csv", index=False)

print("🔥 Saved static embeddings.csv")
print("Shape:", static_embeddings.shape)


🔥 Saved static embeddings.csv
Shape: (8274, 32)


In [8]:
import joblib
import torch

# --- Save preprocessing objects ---
joblib.dump(encoder, "encoder.pkl")
joblib.dump(scaler, "scaler.pkl")




['scaler.pkl']